In [20]:
import pandas as pd
import math 
#!pip install textblob
from textblob import TextBlob as tb

In [21]:
news=pd.read_csv('/Users/chelseayang/Desktop/Paradigm/New_marked_news.csv')
news.drop(['Unnamed: 0'],axis=1)

,author,contents,description,publisher,source_url,title,date,time,Open,High,...,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel,Date_x,Date_y,Mark
0,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,With Israel’s Finance Ministry and the Bank of...,Bitcoinist.com,http://bitcoinist.com/kosher-crypto-bitcoen-se...,Kosher Crypto BitCoen Is Setting a Course for ...,2018-02-02,00:00:08,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,96,2018-02-02,2018-02-02,1.0
1,Michelle Fox,var postloadfunctions var foresee enabled var ...,"Bitcoin may still drop to $7,500, but it will ...",CNBC,https://www.cnbc.com/2018/02/01/bitcoin-near-b...,"Bitcoin near bottom, will rally to $20,000 thi...",2018-02-02,00:02:00,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,146,2018-02-02,2018-02-02,1.0
2,Scott Scanlon,at core cryptocurrency networks miners people ...,At the core of most cryptocurrency networks ar...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/brai...,Brain Genius Submerges His Bitcoin Mining Rig ...,2018-02-02,00:03:08,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,1181,2018-02-02,2018-02-02,1.0
3,Bruce Kleinman,demons digital gold part if already done pleas...,"Demons in Digital Gold, Part 5",Hackernoon.com,https://hackernoon.com/remediation-wherefore-a...,"Remediation, wherefore art thou?",2018-02-02,00:18:34,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,452,2018-02-02,2018-02-02,1.0
4,Jason Murphy,email password remember me feb while systems u...,While the systems underpinning bitcoin are tru...,Crikey.com.au,https://www.crikey.com.au/2018/02/02/cryptotra...,Cryptotragedy: what if bitcoin’s greatest stre...,2018-02-02,00:25:09,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,220,2018-02-02,2018-02-02,1.0
5,CoinTelegraph By Molly Jane Zuckerman,cointelegraph jordan belfort wolf wall street ...,"Jordan Belfort, the “Wolf of Wall Street”, cal...",Cointelegraph.com,https://cointelegraph.com/news/wolf-of-wall-st...,Wolf Of Wall Street Says Bitcoin Could Hit $50...,2018-02-02,00:49:21,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,199,2018-02-02,2018-02-02,1.0
6,Scott Scanlon,timothy b lee pm bitcoin month long price slid...,"Timothy B. Lee – 2/1/2018, 8:15 PM Bitcoin’s m...",Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/more...,More bad news pushes bitcoin’s value below $9000,2018-02-02,01:03:11,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,1181,2018-02-02,2018-02-02,1.0
7,King5.com,seattle weather summary degrees bitcoin droppe...,"Bitcoin drops below key $9,000 level on Coinbase",King5.com,http://www.king5.com/article/news/nation-now/b...,"Bitcoin drops below key $9,000 level on Coinbase",2018-02-02,01:40:17,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,584,2018-02-02,2018-02-02,1.0
8,Jonathan Berr,bitcoin dropped friday reflecting plunge almos...,Bitcoin has lost almost 60 percent of its valu...,CBS News,https://www.cbsnews.com/news/cryptocurrency-pr...,Cryptocurrency prices plunge as regulators cla...,2018-02-02,01:42:56,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,145,2018-02-02,2018-02-02,1.0
9,Duncan Riley,home news duncan riley updated est february th...,The price of bitcoin plunged in trading Thursd...,Siliconangle.com,https://siliconangle.com/blog/2018/02/01/bitco...,Bitcoin price plunges as India announces ban o...,2018-02-02,02:02:58,8547.864403,8562.224306,...,35.615632,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,906,2018-02-02,2018-02-02,

## Clean Description

In [51]:
# Clean words in description (delete stopwords, lower etc.)
import re
from nltk.corpus import stopwords
for i in news.index:
    text = news.loc[i, 'description']
    if pd.isnull(text):
        pass
    else:
        text = re.sub(r"[^a-zA-Z]", " ", text)
        text = [word for word in text.split() if not word in stopwords.words('english')]
        text = (' '.join(text))
        text = text.lower()
        news.loc[i, 'description'] = text

In [70]:
# Convert all the words in description into splittable ones (exclude float or nan)
def converter(x):
    try:
        return ' '.join([x.lower() for x in str(x).split() if x not in stopwords.words('english')])
    except AttributeError:
        return None  # or some other value

news['description'] = news['description'].apply(converter)

## Calculate TF-IDF

In [71]:
def tf(word, blob):
    return word.count(word) / len(blob.split())

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.split())

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [95]:
import numpy as np
news['tfidf']=np.ones(len(news),np.int)

bloblist = news['description']
for i, blob in enumerate(bloblist):
    score_news=0
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.split()}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:5]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
        score_news=score_news+score
    avg_score=round(score_news/5,5)  
    print(avg_score)
    news.iloc[i,-1]=avg_score


Top words in document 1
	Word: bitcoen, TF-IDF: 0.35543
	Word: fill, TF-IDF: 0.32529
	Word: objectives, TF-IDF: 0.32529
	Word: sponsored, TF-IDF: 0.29515
	Word: initiative, TF-IDF: 0.27753
0.31574
Top words in document 2
	Word: neu, TF-IDF: 0.58392
	Word: ner, TF-IDF: 0.58392
	Word: ran, TF-IDF: 0.4849
	Word: predicts, TF-IDF: 0.46896
	Word: rally, TF-IDF: 0.3736
0.49906
Top words in document 3
	Word: mathematical, TF-IDF: 0.35543
	Word: equations, TF-IDF: 0.35543
	Word: specialized, TF-IDF: 0.32529
	Word: performing, TF-IDF: 0.29003
	Word: core, TF-IDF: 0.26502
0.31824
Top words in document 4
	Word: demons, TF-IDF: 2.04371
	Word: gold, TF-IDF: 1.17728
	Word: part, TF-IDF: 1.05351
	Word: digital, TF-IDF: 0.60596
0.97609
Top words in document 5
	Word: ceiling, TF-IDF: 0.34062
	Word: cryptotragedy, TF-IDF: 0.34062
	Word: strength, TF-IDF: 0.34062
	Word: crikey, TF-IDF: 0.34062
	Word: underpinning, TF-IDF: 0.29484
0.33146
Top words in document 6
	Word: jordan, TF-IDF: 0.32372
	Word: belfo

	Word: clickbait, TF-IDF: 0.38928
	Word: poolroom, TF-IDF: 0.38928
	Word: boys, TF-IDF: 0.38928
	Word: betoota, TF-IDF: 0.38928
	Word: skewered, TF-IDF: 0.38928
0.38928
Top words in document 45
	Word: introduce, TF-IDF: 1.24579
	Word: framework, TF-IDF: 1.20696
	Word: efforts, TF-IDF: 1.17445
	Word: regulatory, TF-IDF: 0.80794
	Word: made, TF-IDF: 0.80476
1.04798
Top words in document 46
	Word: ipronto, TF-IDF: 0.32699
	Word: paradise, TF-IDF: 0.32699
	Word: validate, TF-IDF: 0.32699
	Word: incubate, TF-IDF: 0.32699
	Word: allied, TF-IDF: 0.32699
0.32699
Top words in document 47
	Word: coingaming, TF-IDF: 0.31442
	Word: igaming, TF-IDF: 0.31442
	Word: bitcasino, TF-IDF: 0.31442
	Word: sportsbet, TF-IDF: 0.31442
	Word: brands, TF-IDF: 0.28776
0.30909
Top words in document 48
	Word: connecting, TF-IDF: 0.21939
	Word: simplefx, TF-IDF: 0.21396
	Word: downside, TF-IDF: 0.21231
	Word: wave, TF-IDF: 0.20126
	Word: eth, TF-IDF: 0.1909
0.20757
Top words in document 49
	Word: password, TF-IDF: 

	Word: flourish, TF-IDF: 0.30277
	Word: advised, TF-IDF: 0.30277
	Word: rishab, TF-IDF: 0.30277
	Word: hegde, TF-IDF: 0.30277
	Word: ponzicoin, TF-IDF: 0.30277
0.30277
Top words in document 88
	Word: bittrex, TF-IDF: 0.25143
	Word: channel, TF-IDF: 0.24707
	Word: faced, TF-IDF: 0.23641
	Word: sellers, TF-IDF: 0.23345
	Word: tethered, TF-IDF: 0.23345
0.24036
Top words in document 89
	Word: perpetuating, TF-IDF: 0.31442
	Word: disseminating, TF-IDF: 0.31442
	Word: sparked, TF-IDF: 0.2611
	Word: fully, TF-IDF: 0.2455
	Word: self, TF-IDF: 0.23211
0.27351
Top words in document 90
	Word: proactively, TF-IDF: 0.97117
	Word: fm, TF-IDF: 0.90732
	Word: explore, TF-IDF: 0.80876
	Word: government, TF-IDF: 0.46078
	Word: use, TF-IDF: 0.42447
0.7145
Top words in document 91
	Word: preoccupied, TF-IDF: 0.40874
	Word: forgetting, TF-IDF: 0.40874
	Word: grandchildren, TF-IDF: 0.40874
	Word: secre, TF-IDF: 0.40874
	Word: particularly, TF-IDF: 0.36293
0.39958
Top words in document 92
	Word: representing

	Word: shadows, TF-IDF: 0.32372
	Word: emergence, TF-IDF: 0.30244
	Word: drawn, TF-IDF: 0.27795
	Word: opportunities, TF-IDF: 0.27356
	Word: crime, TF-IDF: 0.24681
0.2849
Top words in document 131
	Word: plaything, TF-IDF: 0.40874
	Word: gullible, TF-IDF: 0.40874
	Word: destruction, TF-IDF: 0.40874
	Word: belligerents, TF-IDF: 0.40874
	Word: authenticated, TF-IDF: 0.37408
0.40181
Top words in document 132
	Word: obtain, TF-IDF: 0.4401
	Word: lakh, TF-IDF: 0.37548
	Word: cbdt, TF-IDF: 0.33982
	Word: working, TF-IDF: 0.3347
	Word: notices, TF-IDF: 0.3323
0.36448
Top words in document 133
	Word: must, TF-IDF: 0.74349
	Word: delivered, TF-IDF: 0.69181
	Word: morning, TF-IDF: 0.64265
	Word: every, TF-IDF: 0.62887
	Word: read, TF-IDF: 0.564
0.65416
Top words in document 134
	Word: recommended, TF-IDF: 0.30277
	Word: smokestacks, TF-IDF: 0.30277
	Word: racks, TF-IDF: 0.30277
	Word: massena, TF-IDF: 0.28775
	Word: server, TF-IDF: 0.28775
0.29677
Top words in document 135
	Word: pti, TF-IDF: 0.

	Word: deepening, TF-IDF: 0.31078
	Word: kimchi, TF-IDF: 0.29927
	Word: victim, TF-IDF: 0.27688
	Word: premium, TF-IDF: 0.24382
	Word: overseas, TF-IDF: 0.2391
0.27397
Top words in document 175
	Word: peak, TF-IDF: 1.34287
	Word: half, TF-IDF: 1.09766
	Word: december, TF-IDF: 1.00011
	Word: bitcoin, TF-IDF: 0.19818
0.72777
Top words in document 176
	Word: mavens, TF-IDF: 1.02186
	Word: slang, TF-IDF: 1.02186
	Word: define, TF-IDF: 0.90732
	Word: rundown, TF-IDF: 0.86526
	Word: phenomenon, TF-IDF: 0.83385
0.93003
Top words in document 177
	Word: drops, TF-IDF: 0.97871
	Word: leaving, TF-IDF: 0.97058
	Word: far, TF-IDF: 0.79014
	Word: week, TF-IDF: 0.47369
	Word: currency, TF-IDF: 0.46506
0.73564
Top words in document 178
	Word: bears, TF-IDF: 1.99254
	Word: control, TF-IDF: 1.92565
	Word: taken, TF-IDF: 1.80075
1.14379
Top words in document 179
	Word: braces, TF-IDF: 0.29196
	Word: bbc, TF-IDF: 0.24722
	Word: fr, TF-IDF: 0.23824
	Word: luxembourg, TF-IDF: 0.21769
	Word: leaving, TF-IDF:

...

	Word: cu, TF-IDF: 0.2771
	Word: broke, TF-IDF: 0.21749
	Word: crucial, TF-IDF: 0.20503
	Word: moving, TF-IDF: 0.18507
	Word: bch, TF-IDF: 0.18282
0.2135
Top words in document 6884
	Word: pushed, TF-IDF: 0.53192
	Word: paying, TF-IDF: 0.47771
	Word: fees, TF-IDF: 0.46132
	Word: lower, TF-IDF: 0.42953
	Word: believe, TF-IDF: 0.41815
0.46373
Top words in document 6885
	Word: copywriting, TF-IDF: 0.29196
	Word: bianews, TF-IDF: 0.29196
	Word: kits, TF-IDF: 0.29196
	Word: yield, TF-IDF: 0.27748
	Word: taobao, TF-IDF: 0.2672
0.28411
Top words in document 6886
	Word: jumps, TF-IDF: 1.2949
	Word: hovering, TF-IDF: 1.13142
	Word: much, TF-IDF: 0.65439
	Word: past, TF-IDF: 0.64854
	Word: day, TF-IDF: 0.6172
0.86929
Top words in document 6887
	Word: differ, TF-IDF: 0.34008
	Word: alternatives, TF-IDF: 0.32993
	Word: dive, TF-IDF: 0.30857
	Word: deep, TF-IDF: 0.26925
	Word: explain, TF-IDF: 0.2647
0.30251
Top words in document 6888
	Word: possessed, TF-IDF: 0.51796
	Word: contained, TF-IDF: 0.498

	Word: dilemmas, TF-IDF: 0.29196
	Word: oriented, TF-IDF: 0.25272
	Word: weiss, TF-IDF: 0.25272
	Word: prospects, TF-IDF: 0.24722
	Word: ratings, TF-IDF: 0.23824
0.25657
Top words in document 6928
	Word: docs, TF-IDF: 0.29196
	Word: biddle, TF-IDF: 0.2672
	Word: indulged, TF-IDF: 0.2672
	Word: fantasies, TF-IDF: 0.2672
	Word: paranoiacs, TF-IDF: 0.25923
0.27056
Top words in document 6929
	Word: neutralize, TF-IDF: 0.58392
	Word: streak, TF-IDF: 0.50545
	Word: encouraging, TF-IDF: 0.50545
	Word: winning, TF-IDF: 0.44492
	Word: needed, TF-IDF: 0.43106
0.49416
Top words in document 6930
	Word: tells, TF-IDF: 1.03694
	Word: nine, TF-IDF: 0.88985
	Word: survey, TF-IDF: 0.78097
	Word: americans, TF-IDF: 0.75101
	Word: recent, TF-IDF: 0.48999
0.78975
Top words in document 6931
	Word: cyptocurrency, TF-IDF: 0.30277
	Word: macular, TF-IDF: 0.30277
	Word: degeneration, TF-IDF: 0.30277
	Word: nkorea, TF-IDF: 0.30277
	Word: backtrack, TF-IDF: 0.30277
0.30277
Top words in document 6932
	Word: avoid

	Word: predicts, TF-IDF: 0.43769
	Word: single, TF-IDF: 0.35611
	Word: dorsey, TF-IDF: 0.34527
	Word: jack, TF-IDF: 0.34202
	Word: near, TF-IDF: 0.31714
0.35965
Top words in document 6972
	Word: violated, TF-IDF: 0.3378
	Word: dozen, TF-IDF: 0.31559
	Word: storing, TF-IDF: 0.31559
	Word: rwth, TF-IDF: 0.30766
	Word: aachen, TF-IDF: 0.30766
0.31686
Top words in document 6973
	Word: buzzing, TF-IDF: 0.34062
	Word: entrant, TF-IDF: 0.34062
	Word: proliferation, TF-IDF: 0.30244
	Word: gets, TF-IDF: 0.28286
	Word: recruiting, TF-IDF: 0.28286
0.30988
Top words in document 6974
	Word: clocked, TF-IDF: 0.72586
	Word: breakout, TF-IDF: 0.69221
	Word: bullish, TF-IDF: 0.51544
	Word: earlier, TF-IDF: 0.49362
	Word: litecoin, TF-IDF: 0.43354
0.57213
Top words in document 6975
	Word: sexual, TF-IDF: 0.51847
	Word: imagery, TF-IDF: 0.47648
	Word: hundreds, TF-IDF: 0.40642
	Word: links, TF-IDF: 0.40351
	Word: abuse, TF-IDF: 0.40071
0.44112
Top words in document 6976
	Word: solidity, TF-IDF: 0.68124
	

	Word: valuab, TF-IDF: 0.37158
	Word: securing, TF-IDF: 0.34008
	Word: keys, TF-IDF: 0.30322
	Word: protecting, TF-IDF: 0.30322
	Word: forefront, TF-IDF: 0.29843
0.3233
Top words in document 7016
	Word: snapchat, TF-IDF: 0.74317
	Word: became, TF-IDF: 0.51356
	Word: social, TF-IDF: 0.45812
	Word: media, TF-IDF: 0.41995
	Word: advertising, TF-IDF: 0.38573
0.50411
Top words in document 7017
	Word: emphasize, TF-IDF: 0.34062
	Word: handles, TF-IDF: 0.34062
	Word: streamlining, TF-IDF: 0.34062
	Word: consistently, TF-IDF: 0.30244
	Word: maximize, TF-IDF: 0.29484
0.32383
Top words in document 7018
	Word: participant, TF-IDF: 0.32699
	Word: consecutive, TF-IDF: 0.29927
	Word: wee, TF-IDF: 0.29034
	Word: relief, TF-IDF: 0.28305
	Word: pace, TF-IDF: 0.25212
0.29035
Top words in document 7019
	Word: bec, TF-IDF: 0.34062
	Word: abuzz, TF-IDF: 0.32372
	Word: profound, TF-IDF: 0.31174
	Word: sites, TF-IDF: 0.26596
	Word: image, TF-IDF: 0.25398
0.2992
Top words in document 7020
	Word: quartz, TF-ID

	Word: marketplaces, TF-IDF: 0.88453
	Word: centralized, TF-IDF: 0.76999
	Word: trust, TF-IDF: 0.67528
	Word: issues, TF-IDF: 0.65713
	Word: almost, TF-IDF: 0.56552
0.71049
Top words in document 7059
	Word: thefuture, TF-IDF: 0.28189
	Word: agenda, TF-IDF: 0.28189
	Word: aboard, TF-IDF: 0.28189
	Word: uss, TF-IDF: 0.28189
	Word: hornet, TF-IDF: 0.28189
0.28189
Top words in document 7060
	Word: aap, TF-IDF: 0.29196
	Word: actionable, TF-IDF: 0.29196
	Word: tactics, TF-IDF: 0.27748
	Word: cramer, TF-IDF: 0.25923
	Word: guidance, TF-IDF: 0.24722
0.27357
Top words in document 7061
	Word: evermarkets, TF-IDF: 0.57684
	Word: developing, TF-IDF: 0.47771
	Word: peer, TF-IDF: 0.47771
	Word: futures, TF-IDF: 0.38496
	Word: platform, TF-IDF: 0.32089
0.44762
Top words in document 7062
	Word: priority, TF-IDF: 0.37714
	Word: snowden, TF-IDF: 0.32624
	Word: releases, TF-IDF: 0.32353
	Word: showing, TF-IDF: 0.31847
	Word: nsa, TF-IDF: 0.31166
0.33141
Top words in document 7063
	Word: unlicensed, TF-I

In [96]:
news

,Unnamed: 0,author,contents,description,publisher,source_url,title,date,time,Open,...,Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel,Date_x,Date_y,Mark,tfidf
0,3270,Bitcoinist.net,real time prices vires numeris bitcoin ethereu...,israel finance ministry bank israel considerin...,Bitcoinist.com,http://bitcoinist.com/kosher-crypto-bitcoen-se...,Kosher Crypto BitCoen Is Setting a Course for ...,2018-02-02,00:00:08,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,96,2018-02-02,2018-02-02,1.0,0.31574
1,3271,Michelle Fox,var postloadfunctions var foresee enabled var ...,bitcoin may still drop rally back year early b...,CNBC,https://www.cnbc.com/2018/02/01/bitcoin-near-b...,"Bitcoin near bottom, will rally to $20,000 thi...",2018-02-02,00:02:00,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,146,2018-02-02,2018-02-02,1.0,0.49906
2,3272,Scott Scanlon,at core cryptocurrency networks miners people ...,core cryptocurrency networks miners people use...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/brai...,Brain Genius Submerges His Bitcoin Mining Rig ...,2018-02-02,00:03:08,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,1181,2018-02-02,2018-02-02,1.0,0.31824
3,3273,Bruce Kleinman,demons digital gold part if already done pleas...,demons digital gold part,Hackernoon.com,https://hackernoon.com/remediation-wherefore-a...,"Remediation, wherefore art thou?",2018-02-02,00:18:34,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,452,2018-02-02,2018-02-02,1.0,0.97609
4,3274,Jason Murphy,email password remember me feb while systems u...,systems underpinning bitcoin truly revolutiona...,Crikey.com.au,https://www.crikey.com.au/2018/02/02/cryptotra...,Cryptotragedy: what if bitcoin’s greatest stre...,2018-02-02,00:25:09,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,220,2018-02-02,2018-02-02,1.0,0.33146
5,3275,CoinTelegraph By Molly Jane Zuckerman,cointelegraph jordan belfort wolf wall street ...,jordan belfort wolf wall street calls bitcoin ...,Cointelegraph.com,https://cointelegraph.com/news/wolf-of-wall-st...,Wolf Of Wall Street Says Bitcoin Could Hit $50...,2018-02-02,00:49:21,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,199,2018-02-02,2018-02-02,1.0,0.32372
6,3276,Scott Scanlon,timothy b lee pm bitcoin month long price slid...,timothy b lee pm bitcoin month long price slid...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/more...,More bad news pushes bitcoin’s value below $9000,2018-02-02,01:03:11,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,1181,2018-02-02,2018-02-02,1.0,0.25688
7,3277,King5.com,seattle weather summary degrees bitcoin droppe...,bitcoin drops key level coinbase,King5.com,http://www.king5.com/article/news/nation-now/b...,"Bitcoin drops below key $9,000 level on Coinbase",2018-02-02,01:40:17,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,584,2018-02-02,2018-02-02,1.0,0.77589
8,3278,Jonathan Berr,bitcoin dropped friday reflecting plunge almos...,bitcoin lost almost percent value since decemb...,CBS News,https://www.cbsnews.com/news/cryptocurrency-pr...,Cryptocurrency prices plunge as regulators cla...,2018-02-02,01:42:56,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,145,2018-02-02,2018-02-02,1.0,0.45079
9,3279,Duncan Riley,home news duncan riley updated est february th...,price bitcoin plunged trading thursday india a...,Siliconangle.com,https://siliconangle.com/blog/2018/02/01/bitco...,Bitcoin price plunges as India announces ban o...,2018-02-02,02:02:58,8547.864403,...,300510.207066,8547.594042,12603.493539,433909.334353,2087.647258,906,2018-02-02,2018-02-02,1.0,0.32610


In [97]:
news.to_csv('news_score.csv')